# OCT Applications and Real-World Examples

This final notebook explores how OCT is used in clinical practice and research.

## Learning Objectives

By the end of this notebook, you will understand:
1. OCT in ophthalmology
2. Cardiovascular OCT
3. Dermatology applications
4. Research applications
5. Future directions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Libraries loaded successfully!")

## 1. Ophthalmic OCT (Retinal Imaging)

OCT has revolutionized ophthalmology. It's now the **standard of care** for many retinal diseases.

### Common Applications:

#### Age-Related Macular Degeneration (AMD)
- Visualize drusen (deposits under retina)
- Detect fluid accumulation
- Monitor treatment response

#### Diabetic Retinopathy
- Measure retinal thickness
- Detect macular edema
- Track disease progression

#### Glaucoma
- Measure retinal nerve fiber layer (RNFL) thickness
- Assess optic nerve head
- Early detection of damage

### Retinal Layers Visible with OCT:
1. Internal Limiting Membrane (ILM)
2. Retinal Nerve Fiber Layer (RNFL)
3. Ganglion Cell Layer (GCL)
4. Inner Plexiform Layer (IPL)
5. Inner Nuclear Layer (INL)
6. Outer Plexiform Layer (OPL)
7. Outer Nuclear Layer (ONL)
8. External Limiting Membrane (ELM)
9. Photoreceptors (IS/OS junction)
10. Retinal Pigment Epithelium (RPE)
11. Choroid

In [ ]:
# Simulate retinal layers
def simulate_retinal_oct(width=500, pathology=None):
    """
    Simulate OCT B-scan of retina.
    
    pathology options: None, 'amd', 'edema', 'hole'
    """
    height = 400
    image = np.zeros((height, width))
    
    # Define layer depths (in pixels)
    layers = {
        'ILM': 50,
        'RNFL': 60,
        'GCL': 80,
        'IPL': 100,
        'INL': 130,
        'OPL': 150,
        'ONL': 200,
        'ELM': 220,
        'IS/OS': 240,
        'RPE': 260,
        'Choroid': 280
    }
    
    # Add natural curvature (foveal pit)
    fovea_center = width // 2
    for x in range(width):
        dist_from_center = abs(x - fovea_center) / (width / 4)
        curvature = 30 * np.exp(-dist_from_center**2)
        
        for layer_name, base_depth in layers.items():
            depth = int(base_depth + curvature)
            
            # Layer intensity varies
            if layer_name in ['RNFL', 'IS/OS', 'RPE']:
                intensity = 0.9
                thickness = 3
            else:
                intensity = 0.3 + 0.2 * np.random.rand()
                thickness = 2
            
            for dy in range(thickness):
                if 0 <= depth + dy < height:
                    image[depth + dy, x] = intensity
    
    # Add pathology
    if pathology == 'edema':
        # Macular edema: fluid accumulation
        center_x = width // 2
        for x in range(center_x - 50, center_x + 50):
            if 0 <= x < width:
                for y in range(120, 180):
                    image[y, x] *= 0.3  # Dark fluid
    
    elif pathology == 'amd':
        # Drusen: deposits
        for i in range(5):
            x = np.random.randint(100, width - 100)
            size = np.random.randint(10, 30)
            for dx in range(-size, size):
                for dy in range(-size//2, size//2):
                    if 0 <= x + dx < width and 260 <= 265 + dy < height:
                        if dx**2 + (2*dy)**2 < size**2:
                            image[265 + dy, x + dx] = 0.7
    
    elif pathology == 'hole':
        # Macular hole
        center_x = width // 2
        for x in range(center_x - 30, center_x + 30):
            if 0 <= x < width:
                dist = abs(x - center_x)
                depth_loss = int(30 * (1 - dist / 30))
                for y in range(50, 50 + depth_loss):
                    image[y, x] = 0
    
    # Add noise
    noise = np.random.randn(height, width) * 0.05
    image = np.clip(image + noise, 0, 1)
    
    return image

# Generate examples
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

conditions = [
    (None, 'Normal Retina'),
    ('edema', 'Macular Edema'),
    ('amd', 'Age-Related Macular Degeneration'),
    ('hole', 'Macular Hole')
]

for ax, (pathology, title) in zip(axes.flat, conditions):
    image = simulate_retinal_oct(pathology=pathology)
    ax.imshow(image, cmap='gray', aspect='auto')
    ax.set_title(title, fontsize=13, fontweight='bold')
    ax.set_xlabel('Lateral Position', fontsize=11)
    ax.set_ylabel('Depth', fontsize=11)
    ax.set_yticks([50, 150, 260])
    ax.set_yticklabels(['Retinal Surface', 'Inner Layers', 'RPE/Choroid'])

plt.tight_layout()
plt.show()

print("OCT enables visualization of retinal pathology at micrometer scale!")

## 2. Cardiovascular OCT

Intravascular OCT (IV-OCT) images coronary arteries from inside blood vessels.

### Applications:

#### Atherosclerosis Assessment
- Characterize plaque composition
- Identify vulnerable plaques
- Measure fibrous cap thickness

#### Stent Deployment
- Verify stent apposition
- Detect edge dissection
- Optimize stent expansion

#### Advantages:
- Higher resolution than IVUS (ultrasound)
- Can distinguish lipid-rich plaques
- Real-time imaging during procedures

### Technical Considerations:
- Uses 1300 nm wavelength (better blood penetration)
- Requires flushing blood (transparent to OCT light)
- Rotation speed: 100-200 frames/second
- Pullback speed: 10-40 mm/second

In [ ]:
# Simulate coronary artery cross-section
def simulate_coronary_oct():
    """
    Simulate OCT image of coronary artery.
    """
    size = 500
    image = np.zeros((size, size))
    center = size // 2
    
    # Create coordinates
    y, x = np.ogrid[:size, :size]
    r = np.sqrt((x - center)**2 + (y - center)**2)
    theta = np.arctan2(y - center, x - center)
    
    # Lumen (blood-filled space) - dark
    lumen_mask = r < 120
    image[lumen_mask] = 0.1
    
    # Intima (inner layer)
    intima_mask = (r >= 120) & (r < 135)
    image[intima_mask] = 0.7
    
    # Media (muscle layer)
    media_mask = (r >= 135) & (r < 170)
    image[media_mask] = 0.4
    
    # Adventitia (outer layer)
    adventitia_mask = (r >= 170) & (r < 200)
    image[adventitia_mask] = 0.6
    
    # Add plaque (lipid-rich)
    plaque_angle = np.pi / 3
    plaque_width = np.pi / 6
    plaque_mask = (r >= 120) & (r < 160) & \
                  (np.abs(((theta - plaque_angle + np.pi) % (2*np.pi)) - np.pi) < plaque_width)
    image[plaque_mask] = 0.3  # Lipid appears darker
    
    # Add catheter shadow
    catheter_mask = r < 20
    image[catheter_mask] = 1.0
    
    # Add noise
    noise = np.random.randn(size, size) * 0.05
    image = np.clip(image + noise, 0, 1)
    
    return image

# Create figure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Simulate image
image = simulate_coronary_oct()
ax1.imshow(image, cmap='gray')
ax1.set_title('Intravascular OCT\n(Coronary Artery Cross-Section)', 
              fontsize=13, fontweight='bold')
ax1.axis('off')

# Add annotations
center = 250
ax1.annotate('Catheter', xy=(center, center), xytext=(center-100, center-100),
            arrowprops=dict(arrowstyle='->', color='cyan', lw=2),
            fontsize=11, color='cyan', fontweight='bold')
ax1.annotate('Lumen', xy=(center+80, center), xytext=(center+150, center-80),
            arrowprops=dict(arrowstyle='->', color='yellow', lw=2),
            fontsize=11, color='yellow', fontweight='bold')
ax1.annotate('Lipid Plaque', xy=(center+100, center+80), xytext=(center+150, center+150),
            arrowprops=dict(arrowstyle='->', color='red', lw=2),
            fontsize=11, color='red', fontweight='bold')

# Schematic diagram
ax2.text(0.5, 0.9, 'IV-OCT Clinical Use', ha='center', fontsize=14, fontweight='bold',
        transform=ax2.transAxes)
ax2.text(0.1, 0.75, '✓ Plaque characterization', fontsize=12, transform=ax2.transAxes)
ax2.text(0.1, 0.65, '✓ Stent optimization', fontsize=12, transform=ax2.transAxes)
ax2.text(0.1, 0.55, '✓ Vulnerable plaque detection', fontsize=12, transform=ax2.transAxes)
ax2.text(0.1, 0.45, '✓ Fibrous cap measurement', fontsize=12, transform=ax2.transAxes)

ax2.text(0.1, 0.3, 'Technical Specs:', fontsize=12, fontweight='bold', transform=ax2.transAxes)
ax2.text(0.1, 0.2, '• Resolution: 10-15 μm', fontsize=11, transform=ax2.transAxes)
ax2.text(0.1, 0.13, '• Wavelength: 1300 nm', fontsize=11, transform=ax2.transAxes)
ax2.text(0.1, 0.06, '• Speed: 100+ fps', fontsize=11, transform=ax2.transAxes)

ax2.axis('off')

plt.tight_layout()
plt.show()

print("IV-OCT provides cardiologists with unprecedented vessel wall visualization!")

## 3. Dermatology OCT

OCT in dermatology provides non-invasive "optical biopsy" of skin.

### Applications:

#### Skin Cancer Detection
- Distinguish benign vs. malignant lesions
- Assess tumor margins
- Guide biopsy site selection

#### Cosmetic Applications
- Measure skin thickness
- Evaluate anti-aging treatments
- Monitor wound healing

#### Advantages:
- Non-invasive
- Real-time imaging
- Can image through nail (onychoscopy)

### Skin Layers:
1. Stratum corneum (outermost)
2. Epidermis
3. Dermal-epidermal junction
4. Papillary dermis
5. Reticular dermis

## 4. Other Applications

### Dentistry
- Early caries detection
- Periodontal disease assessment
- Restoration evaluation

### Gastroenterology
- Barrett's esophagus screening
- Inflammatory bowel disease
- Polyp characterization

### Art Conservation
- Examine paint layers
- Detect restoration work
- Study varnish thickness

### Materials Science
- Quality control
- Defect detection
- Layer structure analysis

## 5. Current Research and Future Directions

### AI and Machine Learning
- Automated layer segmentation
- Disease classification
- Predictive analytics

### Functional Extensions
- **Doppler OCT**: Blood flow imaging
- **OCT Angiography (OCTA)**: Vasculature without contrast
- **Polarization-sensitive OCT**: Tissue birefringence
- **OCT Elastography**: Mechanical properties

### Hardware Advances
- Megahertz imaging speeds
- Handheld devices
- Smartphone-based OCT
- Multimodal imaging systems

### New Wavelengths
- Short-wave IR (1700 nm): Deeper penetration
- Visible light OCT: Cellular resolution
- Mid-IR: Molecular imaging

In [ ]:
# Timeline of OCT development
timeline = {
    1991: 'First OCT images (retina)',
    1996: 'FDA approval for ophthalmic use',
    2001: 'Spectral-Domain OCT introduced',
    2006: 'Swept-Source OCT developed',
    2010: 'OCT Angiography demonstrated',
    2015: 'AI-based analysis begins',
    2020: 'Handheld devices widely available',
    2025: 'Advanced multimodal systems'
}

fig, ax = plt.subplots(figsize=(14, 8))

years = list(timeline.keys())
milestones = list(timeline.values())

# Plot timeline
ax.plot(years, np.zeros_like(years), 'k-', linewidth=3)

for i, (year, milestone) in enumerate(timeline.items()):
    y_offset = 0.5 if i % 2 == 0 else -0.5
    ax.plot([year, year], [0, y_offset], 'b-', linewidth=2)
    ax.plot(year, y_offset, 'ro', markersize=15)
    
    va = 'bottom' if y_offset > 0 else 'top'
    ax.text(year, y_offset * 1.1, f'{year}\n{milestone}', 
           ha='center', va=va, fontsize=10, fontweight='bold',
           bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))

ax.set_ylim(-1.5, 1.5)
ax.set_xlim(1985, 2030)
ax.set_xlabel('Year', fontsize=13, fontweight='bold')
ax.set_title('OCT Technology Timeline', fontsize=15, fontweight='bold', pad=20)
ax.set_yticks([])
ax.grid(True, axis='x', alpha=0.3)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.tight_layout()
plt.show()

print("OCT has evolved from a research tool to an indispensable clinical technology!")

## 6. Market and Clinical Impact

### Global Market
- OCT market: $1.5+ billion (2024)
- Growth rate: 8-10% annually
- Ophthalmology: ~70% of market
- Cardiovascular: ~20% of market

### Clinical Procedures
- Millions of OCT scans performed annually
- Standard of care in ophthalmology
- Routine in interventional cardiology

### Impact on Healthcare
- Earlier disease detection
- Better treatment monitoring
- Reduced need for invasive procedures
- Improved patient outcomes

## Summary

In this notebook, we've explored:

1. ✅ **Ophthalmic OCT**: Revolutionized retinal disease diagnosis and monitoring

2. ✅ **Cardiovascular OCT**: Enables precise coronary artery assessment

3. ✅ **Dermatology**: Non-invasive optical biopsy of skin

4. ✅ **Emerging applications**: Dentistry, GI, art conservation, materials science

5. ✅ **Future directions**: AI, functional extensions, new technologies

## Key Takeaways

**OCT Success Factors:**
- Non-invasive and safe
- High resolution (1-15 μm)
- Real-time imaging
- Quantitative measurements
- Relatively low cost

**Why OCT Matters:**
- Enables early disease detection
- Improves treatment outcomes
- Reduces healthcare costs
- Continues to evolve rapidly

## Beyond This Course

### Further Learning:
1. Read key OCT papers and reviews
2. Attend OCT conferences (ARVO, SPIE, OSA)
3. Hands-on experience with OCT systems
4. Implement advanced algorithms
5. Explore specific application areas

### Research Opportunities:
- Novel light sources
- Advanced signal processing
- AI/ML applications
- New clinical applications
- Multimodal imaging

## Conclusion

Congratulations on completing this OCT principles course! You now understand:

- The physics of low-coherence interferometry
- How OCT systems work (TD and FD)
- Resolution and image quality factors
- Signal processing techniques
- Real-world applications

OCT continues to be an exciting field with new developments emerging regularly. Whether you're interested in clinical applications, instrumentation, or algorithm development, there are many opportunities to contribute!

---

## Thank you for learning with us!

**Questions? Feedback? Ideas?**

Please share your experience and help improve these materials!